In [3]:
import pandas as pd
import numpy as np
import json
import os
import capstone_json_functions as cjf

In [4]:
directory = "data/kickstarter_json/"
filename = "2022-06-09.json"
file_path = f"{directory}{filename}"
keys_to_keep = ["name",
            "blurb",
            "goal",
            "pledged",
            "state",
            "slug",
            "disable_communication",
            "country",
            "country_displayable_name",
            "currency",
            "currency_trailing_code",
            "deadline",
            "state_changed_at",
            "created_at",
            "launched_at",
            "backers_count",
            "static_usd_rate",
            "usd_pledged",
            "converted_pledged_amount",
            "fx_rate",
            "usd_exchange_rate",
            "current_currency",
            "usd_type",
            "creator",
            "location",
            "category",
            "profile",
            "spotlight",
            #"urls",
            "rewards"]

In [5]:
df = cjf.df_from_json(file_path,keys_to_keep)

TypeError: tuple indices must be integers or slices, not str

In [ ]:

json_df= pd.read_json(file_path,lines=True)

In [ ]:
json_df.drop(["table_id","robot_id", "run_id"],axis=1,inplace=True)

In [ ]:
json_df.head()

In [ ]:
keys_to_keep = ["name",
            "blurb",
            "goal",
            "pledged",
            "state",
            "slug",
            "disable_communication",
            "country",
            "country_displayable_name",
            "currency",
            "currency_trailing_code",
            "deadline",
            "state_changed_at",
            "created_at",
            "launched_at",
            "backers_count",
            "static_usd_rate",
            "usd_pledged",
            "converted_pledged_amount",
            "fx_rate",
            "usd_exchange_rate",
            "current_currency",
            "usd_type",
            "creator",
            "location",
            "category",
            "profile",
            "spotlight",
            "urls",
            "rewards"]

In [ ]:
df = pd.DataFrame()

for idx,row in json_df.iterrows():
    keys_to_pop = []
    this_row = dict(row["data"])
    for key in this_row.keys():
        if key not in keys_to_keep:
            keys_to_pop.append(key)
    for trash_keys in keys_to_pop:
        this_row.pop(trash_keys)
    #print(this_row)
    if dict(this_row["category"])["id"] == 34:
        df_temp = pd.json_normalize(this_row,sep='_')
        df = pd.concat([df,df_temp])
    #print(df_temp.info())


In [ ]:
data = [json.loads(line)
        for line in open(file_path)]

In [ ]:
data_normalized = pd.json_normalize(data,record_path='data',sep='_',record_prefix='data_')

In [ ]:
data_df = pd.DataFrame()
for entry in data:
    data_dict = entry.copy()
    df = pd.DataFrame(data_dict)
    df.drop(["table_id",'run_id','robot_id'],axis=1,inplace=True)
    df = df.T
    if ": 34," in df["category"].to_string():
        data_df = pd.concat([data_df,df])
    else:
        continue

In [ ]:
df = pd.DataFrame()
for line in open(file_path):
    data_temp = json.loads(line)
    df_temp = pd.json_normalize(data_temp,sep='_')
    df = pd.concat([df,df_temp])


In [ ]:
data_df.head()

In [ ]:
data_dict = data[0].copy()


In [ ]:
del data_dict["table_id"]
del data_dict["run_id"]
del data_dict["robot_id"]

In [ ]:
df = pd.DataFrame(data_dict)

In [ ]:
df.drop(["table_id",'run_id','robot_id'],axis=1,inplace=True)

In [ ]:
df = df.T

In [ ]:
df["category"].to_string()

In [ ]:
with open(file_path) as fp:
    json_object = json.load(fp)